In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json
from typing import List, Dict, Tuple
import re
from collections import Counter

class AdvancedRecipeNLP:
    
    def __init__(self, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
        """
        Initialise le service NLP
        
        Args:
            model_name: Nom du modèle sentence-transformers
                       'paraphrase-multilingual-MiniLM-L12-v2' pour français
                       384 dimensions, rapide et efficace
        """
        print(f"🤖 Chargement du modèle NLP: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.embeddings_cache = {}
        print("✅ Modèle chargé")
    
    def generate_recipe_embedding(self, recipe: Dict) -> np.ndarray:
        """
        Génère un embedding vectoriel pour une recette
        
        Args:
            recipe: Dictionnaire contenant les données de la recette
        
        Returns:
            Vecteur numpy de 384 dimensions
        """
        recipe_id = recipe.get('id')
        
        # Vérifier cache
        if recipe_id in self.embeddings_cache:
            return self.embeddings_cache[recipe_id]
        
        # Construire le texte
        text_parts = []
        
        # Titre (poids important)
        if recipe.get('titre'):
            text_parts.append(recipe['titre'])
            text_parts.append(recipe['titre'])  # Dupliquer pour donner plus de poids
        
        # Description
        if recipe.get('description'):
            text_parts.append(recipe['description'])
        
        # Type et cuisine
        if recipe.get('typeRecette'):
            text_parts.append(f"Type: {recipe['typeRecette']}")
        
        if recipe.get('cuisine'):
            text_parts.append(f"Cuisine: {recipe['cuisine']}")
        
        # Ingrédients
        if recipe.get('ingredients'):
            ingredients_text = "Ingrédients: " + ", ".join([
                ing['nom'] if isinstance(ing, dict) else str(ing)
                for ing in recipe['ingredients']
            ])
            text_parts.append(ingredients_text)
        
        # Caractéristiques
        if recipe.get('vegetarien'):
            text_parts.append("végétarien")
        
        if recipe.get('difficulte'):
            text_parts.append(f"Difficulté: {recipe['difficulte']}")
        
        # Combiner tout
        full_text = ". ".join(text_parts)
        
        # Générer l'embedding avec sentence-transformers
        embedding = self.model.encode(full_text, convert_to_numpy=True)
        
        # Mettre en cache
        if recipe_id:
            self.embeddings_cache[recipe_id] = embedding
        
        return embedding
    
    def calculate_similarity(self, recipe1: Dict, recipe2: Dict) -> float:
        """
        Calcule la similarité cosinus entre deux recettes
        
        Returns:
            Score de similarité entre 0 et 1
        """
        emb1 = self.generate_recipe_embedding(recipe1)
        emb2 = self.generate_recipe_embedding(recipe2)
        
        # Reshape pour sklearn
        emb1 = emb1.reshape(1, -1)
        emb2 = emb2.reshape(1, -1)
        
        similarity = cosine_similarity(emb1, emb2)[0][0]
        
        return float(similarity)
    
    def find_similar_recipes(
        self, 
        target_recipe: Dict, 
        candidate_recipes: List[Dict], 
        top_k: int = 10
    ) -> List[Tuple[Dict, float]]:
        """
        Trouve les K recettes les plus similaires
        
        Returns:
            Liste de tuples (recette, score_similarité)
        """
        target_emb = self.generate_recipe_embedding(target_recipe)
        
        similarities = []
        
        for candidate in candidate_recipes:
            # Ne pas comparer avec elle-même
            if candidate.get('id') == target_recipe.get('id'):
                continue
            
            candidate_emb = self.generate_recipe_embedding(candidate)
            
            # Calculer similarité
            target_emb_reshaped = target_emb.reshape(1, -1)
            candidate_emb_reshaped = candidate_emb.reshape(1, -1)
            similarity = cosine_similarity(target_emb_reshaped, candidate_emb_reshaped)[0][0]
            
            similarities.append((candidate, float(similarity)))
        
        # Trier par similarité décroissante
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        return similarities[:top_k]
    
    def semantic_search(
        self, 
        query: str, 
        recipes: List[Dict], 
        top_k: int = 10
    ) -> List[Tuple[Dict, float]]:
        """
        Recherche sémantique: trouve des recettes pertinentes pour une requête
        
        Args:
            query: Requête en langage naturel
                   Ex: "quelque chose de léger et frais pour l'été"
        
        Returns:
            Liste de tuples (recette, score_pertinence)
        """
        print(f"🔍 Recherche sémantique: '{query}'")
        
        # Générer embedding de la requête
        query_emb = self.model.encode(query, convert_to_numpy=True)
        
        scores = []
        
        for recipe in recipes:
            recipe_emb = self.generate_recipe_embedding(recipe)
            
            # Calculer similarité
            query_emb_reshaped = query_emb.reshape(1, -1)
            recipe_emb_reshaped = recipe_emb.reshape(1, -1)
            score = cosine_similarity(query_emb_reshaped, recipe_emb_reshaped)[0][0]
            
            scores.append((recipe, float(score)))
        
        # Trier par pertinence
        scores.sort(key=lambda x: x[1], reverse=True)
        
        print(f"✅ Trouvé {len(scores)} résultats")
        
        return scores[:top_k]
    
    def analyze_sentiment(self, text: str) -> Dict:
        """
        Analyse de sentiment simple basée sur des mots-clés
        Pour une vraie analyse, utiliser un modèle spécialisé
        
        Returns:
            {"score": float, "label": str}
        """
        text_lower = text.lower()
        
        # Mots positifs français
        positive_words = [
            'délicieux', 'excellent', 'parfait', 'super', 'génial', 'adoré',
            'magnifique', 'succulent', 'savoureux', 'top', 'bravo', 'merci',
            'réussi', 'facile', 'rapide', 'bon', 'très bon', 'recommande'
        ]
        
        # Mots négatifs français
        negative_words = [
            'mauvais', 'horrible', 'raté', 'déçu', 'décevant', 'nul',
            'fade', 'sec', 'dur', 'trop', 'pas bon', 'bof', 'moyen',
            'difficile', 'compliqué', 'échec'
        ]
        
        # Compter les occurrences
        positive_count = sum(1 for word in positive_words if word in text_lower)
        negative_count = sum(1 for word in negative_words if word in text_lower)
        
        # Calculer le score
        total = positive_count + negative_count
        
        if total == 0:
            score = 0.0
            label = "Neutre"
        else:
            score = (positive_count - negative_count) / total
            
            if score > 0.5:
                label = "Très positif"
            elif score > 0.2:
                label = "Positif"
            elif score > -0.2:
                label = "Neutre"
            elif score > -0.5:
                label = "Négatif"
            else:
                label = "Très négatif"
        
        return {
            "score": round(score, 2),
            "label": label,
            "positive_words_found": positive_count,
            "negative_words_found": negative_count
        }
    
    def extract_keywords(self, recipe: Dict, top_n: int = 10) -> List[str]:
        """
        Extrait les mots-clés importants d'une recette
        Utilise TF-IDF simplifié
        """
        # Construire le texte
        text_parts = []
        
        if recipe.get('titre'):
            text_parts.append(recipe['titre'])
        
        if recipe.get('description'):
            text_parts.append(recipe['description'])
        
        if recipe.get('ingredients'):
            for ing in recipe['ingredients']:
                if isinstance(ing, dict):
                    text_parts.append(ing.get('nom', ''))
                else:
                    text_parts.append(str(ing))
        
        full_text = " ".join(text_parts).lower()
        
        # Nettoyer
        full_text = re.sub(r'[^\w\s]', ' ', full_text)
        
        # Mots à ignorer (stop words français simplifiés)
        stop_words = {
            'le', 'la', 'les', 'un', 'une', 'des', 'de', 'du', 'et', 'ou',
            'à', 'au', 'aux', 'en', 'pour', 'avec', 'sans', 'dans', 'sur',
            'cette', 'ces', 'ce', 'son', 'sa', 'ses', 'notre', 'nos', 'votre',
            'vos', 'leur', 'leurs', 'qui', 'que', 'dont', 'où'
        }
        
        # Extraire les mots
        words = full_text.split()
        words = [w for w in words if len(w) > 3 and w not in stop_words]
        
        # Compter les fréquences
        word_counts = Counter(words)
        
        # Retourner les plus fréquents
        keywords = [word for word, count in word_counts.most_common(top_n)]
        
        return keywords
    
    def cluster_recipes(self, recipes: List[Dict], n_clusters: int = 5) -> Dict:
        """
        Regroupe les recettes en clusters basés sur leur similarité
        Utilise K-means sur les embeddings
        """
        from sklearn.cluster import KMeans
        
        print(f"🔬 Clustering de {len(recipes)} recettes en {n_clusters} groupes")
        
        # Générer tous les embeddings
        embeddings = []
        recipe_ids = []
        
        for recipe in recipes:
            emb = self.generate_recipe_embedding(recipe)
            embeddings.append(emb)
            recipe_ids.append(recipe.get('id'))
        
        embeddings_array = np.array(embeddings)
        
        # K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(embeddings_array)
        
        # Organiser par cluster
        clusters = {i: [] for i in range(n_clusters)}
        
        for idx, recipe in enumerate(recipes):
            cluster_id = int(cluster_labels[idx])
            clusters[cluster_id].append({
                'id': recipe.get('id'),
                'titre': recipe.get('titre'),
                'type': recipe.get('typeRecette')
            })
        
        print("✅ Clustering terminé")
        
        return {
            'n_clusters': n_clusters,
            'clusters': clusters,
            'cluster_sizes': {i: len(clusters[i]) for i in range(n_clusters)}
        }
    
    def generate_recipe_matrix(self, recipes: List[Dict]) -> Tuple[np.ndarray, List]:
        """
        Génère la matrice complète d'embeddings pour toutes les recettes
        Utile pour l'analyse globale
        
        Returns:
            (matrice_embeddings, liste_ids)
        """
        print(f"📊 Génération matrice pour {len(recipes)} recettes")
        
        embeddings = []
        recipe_ids = []
        
        for recipe in recipes:
            emb = self.generate_recipe_embedding(recipe)
            embeddings.append(emb)
            recipe_ids.append(recipe.get('id'))
        
        matrix = np.array(embeddings)
        
        print(f"✅ Matrice générée: {matrix.shape}")
        
        return matrix, recipe_ids
    
    def save_embeddings(self, filepath: str = 'recipe_embeddings.npz'):
        """
        Sauvegarde les embeddings en cache
        """
        if not self.embeddings_cache:
            print("⚠️ Aucun embedding en cache")
            return
        
        # Convertir en arrays
        ids = list(self.embeddings_cache.keys())
        embeddings = np.array(list(self.embeddings_cache.values()))
        
        # Sauvegarder
        np.savez_compressed(filepath, ids=ids, embeddings=embeddings)
        
        print(f"💾 {len(ids)} embeddings sauvegardés dans {filepath}")
    
    def load_embeddings(self, filepath: str = 'recipe_embeddings.npz'):
        """
        Charge les embeddings depuis un fichier
        """
        try:
            data = np.load(filepath, allow_pickle=True)
            ids = data['ids']
            embeddings = data['embeddings']
            
            # Reconstruire le cache
            self.embeddings_cache = {
                int(id_): emb for id_, emb in zip(ids, embeddings)
            }
            
            print(f"✅ {len(ids)} embeddings chargés depuis {filepath}")
            
        except FileNotFoundError:
            print(f"⚠️ Fichier {filepath} non trouvé")
    
    def get_cache_stats(self) -> Dict:
    # On essaie de récupérer le nom du modèle, sinon on met une valeur par défaut
        model_name = getattr(self.model, "model_name_or_path", "paraphrase-multilingual-MiniLM-L12-v2")
    
        return {
            'total_embeddings': len(self.embeddings_cache),
            'model_name': model_name,
            'embedding_dimension': self.model.get_sentence_embedding_dimension()
        }


# Exemple d'utilisation
if __name__ == "__main__":
    
    print("=" * 80)
    print("DÉMONSTRATION NLP - SENTENCE TRANSFORMERS")
    print("=" * 80)
    
    # Installer les dépendances
    print("\n📦 Installation des dépendances...")
    print("pip install sentence-transformers scikit-learn numpy pandas")
    
    # Initialiser le service
    nlp = AdvancedRecipeNLP()
    
    # Exemple de recettes
    recipes = [
        {
            'id': 1,
            'titre': 'Pasta Carbonara',
            'description': 'Plat italien traditionnel avec des pâtes, œufs et pancetta',
            'typeRecette': 'plat',
            'cuisine': 'italienne',
            'ingredients': ['pâtes', 'œufs', 'pancetta', 'parmesan'],
            'vegetarien': False,
            'difficulte': 'MOYEN'
        },
        {
            'id': 2,
            'titre': 'Salade César',
            'description': 'Salade fraîche avec laitue, poulet grillé et croûtons',
            'typeRecette': 'entree',
            'cuisine': 'americaine',
            'ingredients': ['laitue', 'poulet', 'croûtons', 'parmesan'],
            'vegetarien': False,
            'difficulte': 'FACILE'
        },
        {
            'id': 3,
            'titre': 'Risotto aux champignons',
            'description': 'Risotto crémeux aux champignons de saison',
            'typeRecette': 'plat',
            'cuisine': 'italienne',
            'ingredients': ['riz arborio', 'champignons', 'parmesan', 'bouillon'],
            'vegetarien': True,
            'difficulte': 'MOYEN'
        }
    ]
    
    print("\n" + "=" * 80)
    print("TEST 1: Similarité entre recettes")
    print("=" * 80)
    
    similarity = nlp.calculate_similarity(recipes[0], recipes[2])
    print(f"\nSimilarité Carbonara ↔ Risotto: {similarity:.2f}")
    print("(Les deux sont italiennes et contiennent du parmesan)")
    
    similarity = nlp.calculate_similarity(recipes[0], recipes[1])
    print(f"Similarité Carbonara ↔ Salade: {similarity:.2f}")
    print("(Moins similaires - cuisines et types différents)")
    
    print("\n" + "=" * 80)
    print("TEST 2: Recherche sémantique")
    print("=" * 80)
    
    query = "un plat italien avec du fromage"
    results = nlp.semantic_search(query, recipes, top_k=3)
    
    print(f"\nRequête: '{query}'")
    print("\nRésultats:")
    for recipe, score in results:
        print(f"  {score:.2f} - {recipe['titre']}")
    
    print("\n" + "=" * 80)
    print("TEST 3: Analyse de sentiment")
    print("=" * 80)
    
    comments = [
        "Délicieux! Mes enfants ont adoré cette recette!",
        "Un peu déçu, c'était trop salé",
        "Correct, rien d'exceptionnel"
    ]
    
    for comment in comments:
        sentiment = nlp.analyze_sentiment(comment)
        print(f"\n'{comment}'")
        print(f"  → {sentiment['label']} (score: {sentiment['score']})")
    
    print("\n" + "=" * 80)
    print("TEST 4: Extraction de mots-clés")
    print("=" * 80)
    
    keywords = nlp.extract_keywords(recipes[0], top_n=5)
    print(f"\nMots-clés pour '{recipes[0]['titre']}':")
    print(f"  {', '.join(keywords)}")
    
    print("\n" + "=" * 80)
    print("TEST 5: Clustering")
    print("=" * 80)
    
    # Ajouter plus de recettes pour le clustering
    all_recipes = recipes * 5  # Simuler plus de recettes
    
    clusters = nlp.cluster_recipes(all_recipes, n_clusters=3)
    
    print(f"\nRecettes regroupées en {clusters['n_clusters']} clusters:")
    for cluster_id, size in clusters['cluster_sizes'].items():
        print(f"  Cluster {cluster_id}: {size} recettes")
    
    print("\n" + "=" * 80)
    print("✅ DÉMONSTRATION TERMINÉE")
    print("=" * 80)
    
    # Statistiques
    stats = nlp.get_cache_stats()
    print(f"\nStatistiques:")
    print(f"  Modèle: {stats['model_name']}")
    print(f"  Dimension: {stats['embedding_dimension']}")
    print(f"  Embeddings en cache: {stats['total_embeddings']}")

DÉMONSTRATION NLP - SENTENCE TRANSFORMERS

📦 Installation des dépendances...
pip install sentence-transformers scikit-learn numpy pandas
🤖 Chargement du modèle NLP: paraphrase-multilingual-MiniLM-L12-v2


Loading weights: 100%|█| 199/199 [00:00<00:00, 1044.25it/s, Materializing param=
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Modèle chargé

TEST 1: Similarité entre recettes

Similarité Carbonara ↔ Risotto: 0.47
(Les deux sont italiennes et contiennent du parmesan)
Similarité Carbonara ↔ Salade: 0.51
(Moins similaires - cuisines et types différents)

TEST 2: Recherche sémantique
🔍 Recherche sémantique: 'un plat italien avec du fromage'
✅ Trouvé 3 résultats

Requête: 'un plat italien avec du fromage'

Résultats:
  0.53 - Pasta Carbonara
  0.41 - Salade César
  0.37 - Risotto aux champignons

TEST 3: Analyse de sentiment

'Délicieux! Mes enfants ont adoré cette recette!'
  → Très positif (score: 1.0)

'Un peu déçu, c'était trop salé'
  → Très négatif (score: -1.0)

'Correct, rien d'exceptionnel'
  → Neutre (score: 0.0)

TEST 4: Extraction de mots-clés

Mots-clés pour 'Pasta Carbonara':
  pâtes, œufs, pancetta, pasta, carbonara

TEST 5: Clustering
🔬 Clustering de 15 recettes en 3 groupes
✅ Clustering terminé

Recettes regroupées en 3 clusters:
  Cluster 0: 5 recettes
  Cluster 1: 5 recettes
  Cluster 2: 5 rec